# Raft

- [The Raft Consensus Algorithm - Web](https://raft.github.io/)
- In Search of an Understandable Consensus Algorithm (Extended Version)

reference implementation:
- C++
	- [LogCabin](https://github.com/logcabin/logcabin)
- Java
	- [Apache Ratis™](https://github.com/apache/ratis)
	- [SOFAJRaft](https://github.com/sofastack/sofa-jraft)

# Quick Start

* [ref](https://zhuanlan.zhihu.com/p/32052223)

基础: 多副本状态机的日志复制.
* 需要保证每个状态机以相同的顺序执行相同的命令.

分解一致性问题为多个子问题:
* Leader选举: Leader election
* 日志复制: log replication
* 安全性: safety
* 日志压缩: log compaction
* 成员变更: membership change

术语:
- term: 任期. 将时间划分为一个个任期. 每个任期的开始都是Leader选举.
  - Leader选举成功: Leader在整个任期内管理.
  - Leader选举失败: 该任期因没有Leader而结束.
- heartbeat: 心跳
- agreement: 协定, 意见一致, 同意

角色:
- Leader: 领导者.
  - 接受客户端请求
  - 向Follower同步请求日志, 当日志同步到大多数节点后让Follower提交日志
- Follower: 跟从者
  - 接受并持久化Leader同步的日志, 在Leader告知可以提交后提交日志.
- Candidate: 候选人
  - Leader选举过程中的临时角色

约束: 任意时刻最多只能有一个Leader, 正常工作时只有Leader和Follower.

## Leader选举

- 初始化时均为Follower.
  - 如果有Leader, Leader会向所有Follower周期性的发送heartbeat以维持其统治.
  - 如果在**选举超时时间**内, 没有收到Leader的heartbeat, 等待随机的一段时间后, 将当前term+1并转换为Candidate, 发起一次Leader选举.
- Candidate发起Leader选举: 给自己投票, 并给集群中的其他节点发送RequestVote.
  - 情况(1): 赢得了多数的选票, 成为Leader. - **选举安全性**
    - 约束: **选举约束**
  - 情况(2): 收到Leader的heartbeat, 有其他节点当选了Leader.
    - 如果该Leader的term >= 当前term: 合法的Leader, 切换到Follower状态.
    - 如果该Leader的term < 当前term, 拒绝处理heartbeat, 保持在Candidate状态.
  - 情况(3): 没有节点赢得多数的选票, Leader选举失败, 等待选举超时时间过后发起下一次选举.

数据结构: heartbeat
* 不带日志条目的AppendEntries.

过程: 投票vote
* 每个节点在指定任期内最多只投给一个Candidate, 按照FIFS(先到先服务)顺序.

保证: 选举出的Leader上一定具有最新的已提交日志.

## 日志复制

数据结构: 日志, 日志条目
* 日志由有序编号的日志条目组成: log index
* 每个日志条目包含: 创建时的term, 用于状态机执行的命令.
  * 一个日志条目被复制到大多数节点上, 这时Leader决定将它应用到状态机中是安全的, 则被认为**可以提交committed**.

过程:
- Leader接受客户端的请求, 将请求作为日志条目Log Entry, 加入到日志中. - **日志只追加**
- Leader并行的向其他节点发送AppendEntries以复制日志条目, 其他节点需要返回确认.
  - 未返回确认的节点, Leader会无限次重试AppendEntries直到所有节点均返回确认.
- 当该日志条目被复制到大多数节点上时, Leader将它**应用到状态机中**, 并向客户端返回执行结果.
  - Leader记录**最新的已提交的日志条目commit index**, 并包含在之后的AppendEntries和heartbeat中. 一旦Follower知道日志条目已提交, 会将它应用到自己的状态机中.


保证: - **日志匹配**
* 如果不同日志中的两个条目有相同的log index和term, 则它们存储的命令是相同的.
  * Leader在一个term内在一个log index上最多创建一个日志条目, 该日志条目在日志中的位置不会改变.
* 如果不同日志中的两个条目有相同的log index和term, 则它们之前的所有条目都是相同的.
  * **AppendEntries的一致性检查**: Leader会将紧邻新日志条目之前的条目的log index和term都包含在RPC请求中, 如果Follwer没有在日志中找到对应的log index和term都相同的日志条目, 则拒绝新的日志条目.
  * **强制复制**: Leader找到Follower与它日志一致的地方(使用AppendEntries调用失败从后往前试), 然后覆盖Follower在该位置之后的条目.
    * Leader维护每个Follower的nextIndex: 下一个发送的日志条目的log index. 初始值为它的下一个日志条目.

## 安全性

* 选举安全性(Election Safety)
  * 在指定term内, 最多选举出一个Leader
* Leader只追加(Leader Append-Only)
  * Leader从不覆盖或删除它的日志条目, 只追加新的条目.
* 日志匹配(Log Matching)
  * 如果两个日志中有term和log index相同的条目, 则这两个条目之前的日志条目均对应相同.
* Leader完备性(Leader Completeness)
  * 如果日志条目在指定term内被提交, 则该条目在所有后续term的Leader的日志中存在.
* 状态机安全性(State Machine Safety)
  * 如果一个服务器已经在特定log index上将日志条目应用到状态机中, 则其它服务器不会在相同的log index上应用不同的日志条目.

机制:
* **选举约束**: 拥有最新的已提交的日志条目的Follower才有资格成为Leader.
  * Candidate发送RequestVote时, 要带上自己的最后一条日志的term和log index.
  * 其他节点收到RequestVote后, 如果发现自己的日志比请求中携带的更新, 则拒绝投票.
* **提交之前term的日志条目**: Leader只能推进commit index来提交当前term的已经复制到大多数节点的日志, **旧term日志的提交**要等到提交当前term的日志来间接提交.
  * 否则会出现已提交的旧任期的日志条目被覆盖的情况. 例: paper Figure 8.

数据操作: 日志条目比较
* 如果term不相同: term大的更新; 否则
* log index大的更新.


## 成员变更

数据结构: 集群配置(cluster configuration)
* 参与共识算法的服务器/节点/成员的集合

问题描述: 
* 集群运行过程中, 参与共识算法的服务器发生变化, 例如替换失败的服务器或修改副本数量等.
* 特殊性: 在成员变更的一致性达成的过程中, 参与投票的节点也可能会发生变化. 
  * 例: paper Figure 10. 集群由3个节点增长为5个节点, 存在同一个任期中选举出两个Leader的情况, 一个有旧配置Cold的大多数支持, 一个有新配置Cnew的大多数支持.

抽象: 联合共识(joint consensus)
* 集群先切换到联合共识这个**过渡性的配置**; 一旦联合共识达成, 切换到新配置.
* 联合共识组合了新旧配置:
  * 日志条目复制到新旧配置中的所有服务器上;
  * 新旧配置中任意服务器可以成为Leader;
  * 协定(选举和日志条目提交)要求新旧配置中 **独立的大多数(seperate majorities)** 支持.

过程:
* Leader收到从Cold到Cnew的配置变更请求, 将联合共识的配置Cold,new作为日志条目存储, 并复制该日志条目.
  * 一旦服务器将Cold,new添加到日志中, 就会在决策中使用该配置.
* 一旦Cold,new被提交, Leader完备性属性保证只有复制了Cold,new日志条目的服务器被选举为Leader.
  * **复制到Cold的大多数节点**, 且**复制到Cnew的大多数节点**时, 提交Cold,new.
* Leader创建Cnew日志条目, 并在集群中复制.
* 一旦Cnew被提交, 不在新配置中的服务器可以关闭.
  * 复制到Cnew的大多数节点, 提交Cnew.
* Leader响应成员变更结果.

保证: 不会同时出现分别使用Cold和Cnew做决策的时间点. 

## 日志压缩
* 避免日志无限增长.
* 每个节点独立的对日志执行snapshot, 只能对已提交的日志条目进行snapshot, snapshot之前的日志都可以丢弃.
* Follower日志落后太多/新加节点: Leader发送InstalledSnapshot.

数据结构: Snapshot
* 日志元数据: 最后一条已提交的日志条目的term和log index. 用于snapshot之后的第一条日志条目的AppendEntries的完整性检查.
* 系统当前状态.

# Paper

```
1 Introduction
2 Replicated state machines
3 What’s wrong with Paxos?
4 Designing for understandability
5 The Raft consensus algorithm
5.1 Raft basics
5.2 Leader election
5.3 Log replication
5.4 Safety
5.4.1 Election restriction
5.4.2 Committing entries from previous terms
5.4.3 Safety argument
5.5 Follower and candidate crashes
5.6 Timing and availability
6 Cluster membership changes
7 Log compaction
8 Client interaction
9 Implementation and evaluation
9.1 Understandability
9.2 Correctness
9.3 Performance
10 Related work
11 Conclusion
```

Raft implements consensus by first electing a distinguished leader, then giving the leader complete responsibility for managing the replicated log.

Given the leader approach, Raft decomposes the consensus problem into three relatively independent subproblems, which are discussed in the subsections that follow:

- **Leader election**: a new leader must be chosen when an existing leader fails (Section 5.2).
- **Log replication**: the leader must accept log entries from clients and replicate them across the cluster, forcing the other logs to agree with its own (Section 5.3).
- **Safety**: the key safety property for Raft is the *State Machine Safety Property* in Figure 3: if any server has applied a particular log entry to its state machine, then no other server may apply a different command for the same log index. Section 5.4 describes how Raft ensures this property; the solution involves an additional restriction on the election mechanism described in Section 5.2.

Figure 2: A condensed summary of the Raft consensus algorithm (excluding membership changes and log compaction). The server behavior in the upper-left box is described as a set of rules that trigger independently and repeatedly. Section numbers such as §5.2 indicate where particular features are discussed. 
A formal specification [31] describes the algorithm more precisely.

Figure 3: Raft guarantees that each of these properties is true at all times. The section numbers indicate where each property is discussed.

- **Election Safety**: *at most one leader* can be elected in a given term. §5.2
- **Leader Append-Only**: a leader never overwrites or deletes entries in its log; it only *appends* new entries. §5.3
- **Log Matching**: if two logs contain an entry with the same index and term, then the logs are identical in all entries *up through* the given index. §5.3
- **Leader Completeness**: if a log entry is *committed* in a given term, then that entry will be present in the logs of the *leaders for all higher-numbered terms*. §5.4
- **State Machine Safety**: if a server has *applied a log entry at a given index to its state machine*, no other server will ever apply a different log entry for the same index. §5.4.3
